In [ ]:
# %pylab nbagg
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns

from lab_repo.classes.dbclasses import dbExperimentSet, dbExperiment
from lab_repo.classes.classes import pcExperimentGroup

import numpy as np
import scipy
from scipy.stats import pearsonr, binned_statistic, sem, wilcoxon, f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.api as sm

import pandas as pd

from datetime import datetime

import lab_repo.analysis.dendrites_analysis as da
from lab_repo.misc.plotting import cdf
import lab_repo.analysis.behavior_analysis as ba

import cPickle as pkl
import os

import lab_repo.misc.plotting as lpl

from matplotlib import rc

import lab_repo.analysis.reward_analysis as ra
import lab_repo.analysis.place_cell_analysis as pca

# Set rc-params
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['ytick.labelsize'] = 6
plt.rcParams['xtick.labelsize'] = 6
plt.rcParams['axes.labelsize'] = 7
plt.rcParams['boxplot.boxprops.linewidth'] = 1

rc('font',**{'family':'sans-serif','sans-serif':['helvetica', 'sans-serif']})
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
def clean(ax, full=False, offset=0):
    if full:
        sns.despine(ax=ax, left=True, bottom=True, offset=offset)
        ax.set_yticks([])
        ax.set_xticks([])
    else:
        sns.despine(ax=ax, offset=offset)

In [ ]:
tids = [13897, 13886, 13888, 13912, 13898, 13889, 13907, 13882, 16162, 16141, 16169,
 16156, 16151, 16160, 16142, 16145, 16173, 16190, 16177, 16182, 16179, 16194,
 16192, 16186, 16208, 16191, 16203, 16180, 16197, 16201, 16207, 16195, 17845,
 17833, 17835, 17851, 17846, 17858, 17839, 17852, 22261, 22249, 22241, 22273,
 18142, 18161, 22283, 22280, 18195, 18192, 22270, 18185, 22262, 22244, 22264,
 22213, 22242, 22246, 22220, 22231, 22238, 22223, 22226, 22228]

pc_kwargs = {'imaging_label': 'mergedmerged',
                 'nPositionBins': 100,
                 'channel': 'Ch2',
                 'demixed': False,
                 'pf_subset': None}

gol_grp = pcExperimentGroup([dbExperiment(x) for x in tids], **pc_kwargs)

plotdir = '/home/sebi/plots/dends/fig4/'

## Plotting

In [ ]:
colors = sns.xkcd_palette(['golden rod', 'pale purple'])
color_dict = {'Close': colors[0], 'Far': colors[1]}

### Lick and Centroid Distance

In [ ]:
fig = plt.figure(figsize=(1, 1.5), dpi=200)
ax = fig.add_subplot(111)
for s, c in zip([1, 2, 3, 4], ['0.6', '0.4', '0.2', '0']):
    cdf(ax, lick_dist.loc[lick_dist['session'] == s]['value'], color=c, label='RZ 1', ls='--', lw=1)
for s, c in zip([1, 2, 3, 4], ['0.6', '0.4', '0.2', '0']):
    cdf(ax, lick_dist.loc[lick_dist['session'] == s + 4]['value'], color=c, label='RZ 2', lw=1)

ax.set_xlabel('Lick Distance to RZ\n(Normalized)')
ax.set_ylabel('Cumulative Fraction')
clean(ax)

ax.set_xticks([0, 0.25, 0.5])
ax.set_xlim([0, 0.5])

ax.axvspan(0, 0.5/3., color=colors[0], alpha=0.5, lw=0)
ax.axvspan(1/3., 0.5, color=colors[1], alpha=0.5, lw=0)

fig.savefig(os.path.join(plotdir, 'lick_dist_by_session.svg'))

In [ ]:
fig = plt.figure(figsize=(1, 1.5), dpi=200)
ax = fig.add_subplot(111)

cdf(ax, dend_pf_dists.loc[dend_pf_dists['day'].isin([1, 2])]['value'], color='k', label='RZ 1', ls='--', lw=1)
cdf(ax, dend_pf_dists.loc[dend_pf_dists['day'].isin([3, 4])]['value'], color='k', label='RZ 2', lw=1)

cdf(ax, pf_dists.loc[pf_dists['day'].isin([1, 2])]['value'], color='#1c75bc', label='RZ 1', ls='--', lw=1)
cdf(ax, pf_dists.loc[pf_dists['day'].isin([3, 4])]['value'], color='#1c75bc', label='RZ 2', lw=1)


ax.set_xlabel('Centroid Distance to RZ\n(Normalized)')
ax.set_ylabel('Cumulative Fraction')
clean(ax, offset=4)

ax.set_xticks([0, 0.25, 0.5])
ax.set_xlim([0, 0.5])

ax.axvspan(0, 0.5/3., color=colors[0], alpha=0.5, lw=0)
ax.axvspan(1/3., 0.5, color=colors[1], alpha=0.5, lw=0)

fig.savefig(os.path.join(plotdir, 'dend_soma_centroid_dist_by_condition.svg'))

In [ ]:
fig = plt.figure(figsize=(1, 1.5), dpi=200)
ax = fig.add_subplot(111)

cdf(ax, dend_pf_dists.loc[dend_pf_dists['day'].isin([1, 2])]['peak_value'], color='k', label='RZ 1', ls='--', lw=1)
cdf(ax, dend_pf_dists.loc[dend_pf_dists['day'].isin([3, 4])]['peak_value'], color='k', label='RZ 2', lw=1)

cdf(ax, pf_dists.loc[pf_dists['day'].isin([1, 2])]['peak_value'], color='#1c75bc', label='RZ 1', ls='--', lw=1)
cdf(ax, pf_dists.loc[pf_dists['day'].isin([3, 4])]['peak_value'], color='#1c75bc', label='RZ 2', lw=1)


ax.set_xlabel('PF Peak Distance to RZ\n(Normalized)')
ax.set_ylabel('Cumulative Fraction')
clean(ax, offset=4)

ax.set_xticks([0, 0.25, 0.5])
ax.set_xlim([0, 0.5])

ax.axvspan(0, 0.5/3., color=colors[0], alpha=0.5, lw=0)
ax.axvspan(1/3., 0.5, color=colors[1], alpha=0.5, lw=0)

fig.savefig(os.path.join(plotdir, 'dend_soma_peak_dist_by_condition.svg'))

In [ ]:
a = pf_dists.loc[pf_dists['day'].isin([1, 2])]['peak_value']
b = pf_dists.loc[pf_dists['day'].isin([3, 4])]['peak_value']

print a.shape, a.mean(), a.sem()
print b.shape, b.mean(), b.sem()

mannwhitneyu(a, b)

In [ ]:
def add_soma(row):
    return row['roi'].split('_')[0]

dend_pf_dists['soma'] = dend_pf_dists.apply(add_soma, axis=1)

In [ ]:
mean_dend_dists = dend_pf_dists.groupby(['condition', 'day', 'expt', 'fov', 'mouse_name', 'session', 'soma'], as_index=False).mean()

In [ ]:
merge_df = activity_dists.merge(tuning_df, left_on=['condition', 'day', 'expt', 'fov', 'mouse_name', 'session', 'roi'],
                                right_on=['condition', 'day', 'expt', 'fov', 'mouse', 'session', 'soma'],
                                suffixes=['_soma', '_dend'])

In [ ]:
a = dend_pf_dists.loc[dend_pf_dists['day'].isin([1, 2])]['peak_value']
b = dend_pf_dists.loc[dend_pf_dists['day'].isin([3, 4])]['peak_value']

print a.shape, a.mean(), a.sem()
print b.shape, b.mean(), b.sem()

mannwhitneyu(a, b)

In [ ]:
# Tuning
# Somatic and Dendritic Tuning by Day
dend_filter = lambda x: '_' in x.label

all_tcs = [[], []]
soma_data = gol_grp.data(roi_filter=dend_filter)
soma_pfs = gol_grp.pfs(roi_filter=dend_filter)

for expt in gol_grp:
           
    shift = 50 - int(expt.rewardPositions()[0] * 100)
    
    all_tcs[expt.condition - 1].extend([np.roll(x, shift) for x, p in zip(soma_data[expt], soma_pfs[expt]) if p])

fig = plt.figure(figsize=(1.5, 1.5), dpi=200)
for i in xrange(1, 3):
    ax = fig.add_subplot(1, 2, i)
    
    hm = np.vstack(all_tcs[i-1])
    maxes = np.argmax(hm, axis=1)
    sort_idx = np.argsort(maxes)
    new_hm = hm[sort_idx, :]
    new_hm = new_hm / np.max(new_hm, axis=1, keepdims=True)
    
    print new_hm.shape
    
    sns.heatmap(new_hm, cbar=False, rasterized=True)
    
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_yticklabels([0, new_hm.shape[0]-1], rotation=0)
    
    ax.axvline(50, color='w', ls='--', lw=1)

# fig.savefig(os.path.join(plotdir, 'dend_place_fields_by_condition_RZ_sort.svg'))

In [ ]:
cell_set = gol_grp._cells

In [ ]:
tuning_df = tc_correlation(cell_set)

In [ ]:
tuning_df['pf_type'] = tuning_df.apply(pf_type, axis=1)

In [ ]:
def dist_grp(x):
    # Split distances into thirds
    if x['value_dist'] <= 0.167:
        return 'Close'
    elif x['value_dist'] > .33:
        return 'Far'
    else:
        return 'Mid'

In [ ]:
fig = plt.figure(figsize=(1.2, 1.5), dpi=200)
ax = fig.add_subplot(111)

mean_df = tuning_df.groupby(['expt', 'soma', 'pf', 'condition'], as_index=False).mean()
mean_df = mean_df.loc[mean_df['pf']]

test_df = mean_df.merge(pf_dists, left_on=['soma', 'expt', 'condition', 'day'], right_on=['roi', 'expt', 'condition', 'day'])

test_df['value_dist'] = test_df['peak_value']
test_df['group'] = test_df.apply(dist_grp, axis=1)

test_df = test_df.loc[test_df['group'] != 'Mid']

# Split by distance group
sns.boxplot(y='abs_dist', x='condition', hue='group', palette=colors, data=test_df, hue_order=['Close', 'Far'],
            showfliers=False, whis=[5, 95])

plt.setp(ax.artists, edgecolor='k', lw=1);
plt.setp(ax.lines, color='k', lw=1);
sns.stripplot(y='abs_dist', x='condition', hue='group', data=test_df, hue_order=['Close', 'Far'],
              palette=colors, edgecolor='k', linewidth=0.5, alpha=0.5, size=2, dodge=True)

ax.get_legend().set_visible(False)

clean(ax, offset=4)
ax.set_ylabel('Mean Dist to Soma Peak\n(Normalized)')
ax.set_xlabel('Condition')

ax.axhline(0.25, lw=1, ls='--', color='0.5')
ax.set_yticks([0, 0.1, 0.2, 0.3, 0.4, 0.5])
fig.savefig(os.path.join(plotdir, 'dend_soma_tuning_dist_by_condition.svg'))

In [ ]:
import statsmodels.api as sm
formula = 'abs_dist ~ C(group) + C(condition) + C(group)*C(condition)'
model = ols(formula, data=test_df).fit()
sm.stats.anova_lm(model, typ=2)

In [ ]:
# Same as above but shift cells to be centered somatic peak'

soma_filter = lambda x: '_' not in x.label
dend_filter = lambda x: '_' in x.label

all_tcs = [[], []]
soma_data = gol_grp.data(roi_filter=soma_filter)
soma_pfs = gol_grp.pfs(roi_filter=soma_filter)

dend_data = gol_grp.data(roi_filter=dend_filter)
dend_pfs = gol_grp.pfs(roi_filter=dend_filter)
bad_dends = []
for expt in gol_grp:
    
    soma_rois = expt.rois(roi_filter=soma_filter)
    soma_labels = [x.label.lower() for x in soma_rois]
    
    shift_dict = {}
    for soma_label, soma_tc in zip(soma_labels, soma_data[expt]):
        shift_dict[soma_label] = 50 - np.argmax(soma_tc)
    
    dend_rois = expt.rois(roi_filter=dend_filter)
    dend_labels = [x.label.lower() for x in dend_rois]
    
    expt_tcs = []
#     print expt.trial_id, expt.day, len([x for x in dend_pfs[expt] if x]), len(dend_pfs[expt])
    for dend_label, dend_tc, dend_pf in zip(dend_labels, dend_data[expt], dend_pfs[expt]):
        if not dend_pf:
            continue
        
        try:
            shift = shift_dict[dend_label.split('_')[0]]
        except KeyError:
            bad_dends.append((expt.trial_id, dend_label))
            continue
        
        expt_tcs.append(np.roll(dend_tc, shift))
    
    
    all_tcs[expt.condition - 1].extend(expt_tcs)

fig = plt.figure(figsize=(1.5, 1.5), dpi=200)
for i in xrange(1, len(all_tcs)+1):
    ax = fig.add_subplot(1, len(all_tcs), i)
    
    if len(all_tcs[i-1]) == 0:
        continue
    
    hm = np.vstack(all_tcs[i-1])
    maxes = np.argmax(hm, axis=1)
    sort_idx = np.argsort(maxes)
    new_hm = hm[sort_idx, :]
    new_hm = new_hm / np.max(new_hm, axis=1, keepdims=True)
    
    print new_hm.shape
    
    sns.heatmap(new_hm, cbar=False, rasterized=True)
    
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_yticklabels([0, new_hm.shape[0]-1], rotation=0)
    
    ax.axvline(50, color='w', ls='--', lw=1)

fig.savefig(os.path.join(plotdir, 'dend_place_fields_by_condition_soma_sort.svg'))

In [ ]:
sens = pca.sensitivity(gol_grp)
spef = pca.specificity(gol_grp)
split_corr = pca.within_session_correlation(gol_grp, n_splits=1)

sens = sens.rename(index=str, columns={'expt_id': 'expt', 'roi':'dend', 'FOV':'fov'})
spef = spef.rename(index=str, columns={'expt_id': 'expt', 'roi':'dend', 'FOV':'fov'})
split_corr = split_corr.rename(index=str, columns={'expt_id': 'expt', 'roi':'dend', 'FOV':'fov'})

In [ ]:
running_freq = running_freq.rename(index=str, columns={'roi':'dend'})

In [ ]:
def pf_type(row):
    
    pfo = row['pf_overlap']
    
    if np.isnan(pfo):
        return 'Soma'
    elif pfo:
        return 'Co-PF'
    else:
        return 'Iso-PF'


merge_df = sens.merge(spef, on=['expt', 'fov', 'dend'], suffixes=['_sens', '_spef'])
merge_df = merge_df.merge(running_freq, on=['expt', 'fov', 'dend'], suffixes=['', '_split'])
merge_df = merge_df.merge(tuning_df, on=['expt', 'fov', 'dend'], suffixes=['', '_corr'], how='left')

merge_df['pf_type'] = merge_df.apply(pf_type, axis=1)

cols = ['value_spef', 'value_sens', 'frequency', 'order']
names = ['Specificity', 'Sensitivity', 'RUN Event Rate (Hz)', 'Branch Order']

colors = ['#ec5e34', '#2bb673', '#2278b5']

for i in xrange(1, 4):
    fig = plt.figure(figsize=(1, 1.5), dpi=200)
    ax = fig.add_subplot(111)
    sns.boxplot(x='pf_type', y=cols[i-1], data=merge_df, whis=[5, 95], showfliers=False,
                order=['Iso-PF', 'Co-PF', 'Soma'], palette=colors)
    plt.setp(ax.artists, edgecolor='k', lw=1);
    plt.setp(ax.lines, color='k', lw=1);
    sns.stripplot(x='pf_type', y=cols[i-1], data=merge_df, edgecolor='k', linewidth=0.5, alpha=0.5,
                  dodge=True, size=2, order=['Iso-PF', 'Co-PF', 'Soma'], palette=colors)
    clean(ax, offset=4)

    ax.set_xlabel('')
#     ax.set_xticklabels(['Iso-PFs', 'Co-PFs'])
    ax.set_ylabel(names[i-1])
    if i in [1, 2]:
        ax.set_yticks([0, 0.2, 0.4, 0.6, 0.8, 1])
    plt.xticks(rotation=45)
    
    fig.savefig(os.path.join(plotdir, 'iso_pf_tuning_quality_{}.svg'.format(cols[i-1])))

In [ ]:
col = 'value_spef'

a = merge_df.loc[merge_df['pf_type'] == 'Iso-PF'][col].dropna()
b = merge_df.loc[merge_df['pf_type'] == 'Co-PF'][col].dropna()
c = merge_df.loc[merge_df['pf_type'] == 'Soma'][col].dropna()

print f_oneway(a, b, c)

vals = merge_df.dropna(subset=[col])

print pairwise_tukeyhsd(endog=vals[col], groups=vals['pf_type'], alpha=0.05)

In [ ]:
col = 'value_sens'

a = merge_df.loc[merge_df['pf_type'] == 'Iso-PF'][col].dropna()
b = merge_df.loc[merge_df['pf_type'] == 'Co-PF'][col].dropna()
c = merge_df.loc[merge_df['pf_type'] == 'Soma'][col].dropna()

print f_oneway(a, b, c)

vals = merge_df.dropna(subset=[col])

print pairwise_tukeyhsd(endog=vals[col], groups=vals['pf_type'], alpha=0.05)

In [ ]:
col = 'frequency'

a = merge_df.loc[merge_df['pf_type'] == 'Iso-PF'][col].dropna()
b = merge_df.loc[merge_df['pf_type'] == 'Co-PF'][col].dropna()
c = merge_df.loc[merge_df['pf_type'] == 'Soma'][col].dropna()

print f_oneway(a, b, c)

vals = merge_df.dropna(subset=[col])

print pairwise_tukeyhsd(endog=vals[col], groups=vals['pf_type'], alpha=0.05)

In [ ]:
merge_df['abs_peak_dist'] = merge_df['peak_dist'].apply(np.abs)

In [ ]:
fig = plt.figure(figsize=(1, 1.5), dpi=200)
ax = fig.add_subplot(111)

lpl.cdf(ax, np.abs(merge_df.loc[(merge_df['pf_type']=='Co-PF') & merge_df['pf']]['peak_dist']), color=colors[1], lw=1);
# lpl.cdf(ax, np.abs(merge_df.loc[(merge_df['pf_type']=='Iso-PF') & merge_df['pf']]['peak_dist']), color=colors[0], lw=1);
# lpl.cdf(ax, np.abs(merge_df.loc[(merge_df['pf_type']=='Iso-PF') & (merge_df['pf']==False)]['peak_dist']), color='orange', lw=1);

lpl.cdf(ax, np.abs(merge_df.loc[(merge_df['pf_type']=='Iso-PF')]['peak_dist']), color=colors[0], lw=1);

clean(ax, offset=4)
ax.set_xlim([0, 100])
ax.set_xlabel('Distance to Somatic Peak (cm)')
ax.set_ylabel('Cumulative Fraction')

left, bottom, width, height = [0.7, 0.125, 0.2, 0.2]
ax2 = fig.add_axes([left, bottom, width, height])
sns.barplot(y='abs_peak_dist', x='pf_type', palette=[colors[1], colors[0]], order=['Co-PF', 'Iso-PF'], data=merge_df, ci=68, capsize=0.1, errwidth=1,
            dodge=False, edgecolor='0.2', linewidth=1, ax=ax2)
clean(ax2, offset=1)
ax2.set_ylabel('')
ax2.set_xlabel('')
ax2.set_xticks([])
ax2.set_ylim([0, 55])
ax2.set_yticks([0, 25, 50])

# fig.savefig(os.path.join(plotdir, 'dend_pf_peak_distance.svg'))

In [ ]:
wilcoxon(merge_df.loc[merge_df['pf_type'] == 'Co-PF']['abs_peak_dist'] - 48)

In [ ]:
# Tuning
# Somatic and Dendritic Tuning by Day
soma_filter = lambda x: '_' not in x.label

all_tcs = [[], []]
soma_data = gol_grp.data(roi_filter=soma_filter)
soma_pfs = gol_grp.pfs(roi_filter=soma_filter)

for expt in gol_grp:
           
    shift = 50 - int(expt.rewardPositions()[0] * 100)
    
    all_tcs[expt.condition - 1].extend([np.roll(x, shift) for x, p in zip(soma_data[expt], soma_pfs[expt]) if p])

fig = plt.figure(figsize=(1.5, 1.5), dpi=200)
for i in xrange(1, 3):
    ax = fig.add_subplot(1, 2, i)
    
    hm = np.vstack(all_tcs[i-1])
    maxes = np.argmax(hm, axis=1)
    sort_idx = np.argsort(maxes)
    new_hm = hm[sort_idx, :]
    new_hm = new_hm / np.max(new_hm, axis=1, keepdims=True)
    
    print hm.shape
    
    sns.heatmap(new_hm, cbar=False, rasterized=True)
    
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_yticklabels([0, new_hm.shape[0]-1], rotation=0)
    
    ax.axvline(50, color='w', ls='--', lw=1)

# fig.savefig(os.path.join(plotdir, 'place_cells_by_condition.svg'))

### Licktogram

In [ ]:
mouse = 1
count = 0
labels = ['Condition 1\nFirst Session', 'Condition 1\nLast Session', 'Condition 2\nFirst Session', 'Condition 2\nLast Session']
for i in [0, 3, 4, 7]:
    
    fig = plt.figure(figsize=(.75, .75), dpi=200)
    ax = fig.add_subplot(111)
    expt = gol_grp[mouse*8 + i]

    ba.licktogram(expt, ax=ax, plot_belt=False, nPositionBins=100, shade_reward=True);
    clean(ax, offset=1)

    ax.set_xlabel('Normalized Position')
    ax.set_ylabel('Fraction Licks')
    ax.set_title('')
    
    ax.set_ylim([0, 0.25])
    ax.set_yticklabels(['0', '0.1', '0.2'])
        
    count += 1
    
    fig.savefig(os.path.join(plotdir, 'licktogram+{}.svg'.format(count)))

### SWR Recruitment and dend activity

In [ ]:
activity_dists['group'] = activity_dists.apply(dist_grp, axis=1)

In [ ]:
same_session_bsp = gol_running_bsp.groupby(['fov', 'day', 'expt', 'roi', 'session', 'mouse_name'], as_index=False).mean()

merge_df = soma_responses.merge(activity_dists, on=['roi', 'expt', 'condition', 'mouse_name', 'session'], suffixes=['_swr', '_dist'])
merge_df['value_dist'] = merge_df['value']
merge_df['group'] = merge_df.apply(dist_grp, axis=1)

merge_df = merge_df.merge(same_session_bsp, on=['roi', 'condition', 'mouse_name', 'session'], suffixes=['', '_bsp'])

fig = plt.figure(figsize=(1.5, 1.5), dpi=200)
ax = fig.add_subplot(111)

df = merge_df.loc[(merge_df['condition'].isin([1, 2]))]
df['recruited'] = (df['frac_active'] > 0) & (df['n_ripples'] > 0)
df = df.loc[df['group'].isin(['Close', 'Far'])]
sns.scatterplot(y='value_bsp', x='frac_active', hue='group', palette=color_dict,
                hue_order=['Close', 'Far'], data=df, s=9, edgecolor='k', markers=['^', 'o'], alpha=0.75, lw=0.5)

ax.get_legend().set_visible(False)
clean(ax, offset=4)
ax.set_xlabel('SWR Recruitment Fraction')
ax.set_ylabel('Same Session Mean RUN BSP')

fig.savefig(os.path.join(plotdir, 'all_bsp_vs_swr_recruitment.svg'))

In [ ]:
merge_df = merge_df.dropna(subset=['value_bsp', 'frac_active'])
print spearmanr(merge_df['value_bsp'], merge_df['frac_active'])

In [ ]:
merge_df = soma_responses.merge(pf_dists, on=['roi', 'expt', 'condition', 'mouse_name', 'session'], suffixes=['_swr', '_dist'])
# merge_df['value_dist'] = merge_df['value']
merge_df['value_dist'] = merge_df['peak_value']
merge_df['group'] = merge_df.apply(dist_grp, axis=1)

merge_df = merge_df.merge(same_session_bsp, on=['roi', 'condition', 'mouse_name', 'session'], suffixes=['', '_bsp'])

fig = plt.figure(figsize=(1.5, 1.5), dpi=200)
ax = fig.add_subplot(111)

df = merge_df.loc[(merge_df['condition'].isin([1, 2]))]
df['recruited'] = (df['frac_active'] > 0) & (df['n_ripples'] > 0)
df = df.loc[df['group'].isin(['Close', 'Far'])]
sns.scatterplot(y='value_bsp', x='frac_active', hue='group', palette=color_dict, hue_order=['Close', 'Far'],
                data=df, s=7, edgecolor='k', markers=['^', 'o'], alpha=0.75, lw=0.5)
ax.get_legend().set_visible(False)
clean(ax, offset=4)
ax.set_xlabel('SWR Recruitment Fraction')
ax.set_ylabel('Same Session Mean In-PF BSP')

# fig.savefig(os.path.join(plotdir, 'pf_bsp_vs_swr_recruitment.svg'))

In [ ]:
merge_df = merge_df.dropna(subset=['value_bsp', 'frac_active'])
print spearmanr(merge_df['value_bsp'], merge_df['frac_active'])

### Group BSPs

In [ ]:
fig = plt.figure(figsize=(0.8, 1.5), dpi=200)
ax = fig.add_subplot(111)

mean_bsp = gol_running_bsp.groupby(['fov', 'day', 'expt', 'roi', 'session', 'mouse_name'], as_index=False).mean()
merge_df = mean_bsp.merge(activity_dists, on=['fov', 'day', 'expt', 'roi'], suffixes=['', '_dist'])

merge_df['value_dist'] = merge_df['peak_value']
merge_df['group'] = merge_df.apply(dist_grp, axis=1)
df = merge_df.loc[merge_df['condition'] == 2]
sns.boxplot(x='group', y='value', data=df, whis=[5, 95], showcaps=True, showfliers=False, order=['Close', 'Far'], palette=color_dict)
plt.setp(ax.artists, edgecolor='k', lw=1);
plt.setp(ax.lines, color='k', lw=1);
sns.stripplot(x='group', y='value', data=df, order=['Close', 'Far'], palette=color_dict, edgecolor='k', linewidth=0.5, alpha=0.5, size=2)

clean(ax, offset=4)
ax.set_xticklabels(['Near RZ', 'Far'])
ax.set_ylabel('Mean RUN BSP')
ax.set_xlabel('')

ax.set_ylim([0, 0.55])

fig.savefig(os.path.join(plotdir, 'cond_2_mean_run_bsp_peak.svg'))
print mannwhitneyu(df.loc[df['group'] == 'Close']['value'], df.loc[df['group'] == 'Far']['value'])
# print pearsonr(df['value'], df['value_dist'])

In [ ]:
a = df.loc[df['group'] == 'Close']['value'].dropna()
b = df.loc[df['group'] == 'Far']['value'].dropna()

print a.shape, a.mean(), a.sem()
print b.shape, b.mean(), b.sem()

In [ ]:
fig = plt.figure(figsize=(0.8, 1.5), dpi=200)
ax = fig.add_subplot(111)

mean_bsp = gol_swr_bsp.groupby(['fov', 'day', 'expt', 'roi', 'session', 'mouse_name'], as_index=False).mean()
merge_df = mean_bsp.merge(pf_dists, on=['fov', 'day', 'expt', 'roi'], suffixes=['', '_dist'])
merge_df['value_dist'] = merge_df['peak_value']

merge_df['group'] = merge_df.apply(dist_grp, axis=1)
df = merge_df.loc[merge_df['condition'] == 2]
sns.boxplot(x='group', y='value', data=df, whis=[5, 95], showcaps=True, showfliers=False, order=['Close', 'Far'], palette=color_dict)
plt.setp(ax.artists, edgecolor='k', lw=1);
plt.setp(ax.lines, color='k', lw=1);
sns.stripplot(x='group', y='value', data=df, order=['Close', 'Far'], palette=colors, edgecolor='k', linewidth=0.5, alpha=0.5, size=2)
clean(ax, offset=4)
ax.set_xticklabels(['Near RZ', 'Far'])
ax.set_ylabel('Mean In-PF BSP')
ax.set_xlabel('')

# fig.savefig(os.path.join(plotdir, 'dist_group_pf_bsp_peak.svg'))

print df.loc[df['group'] == 'Close']['value'].mean(), df.loc[df['group'] == 'Far']['value'].mean()
print df.loc[df['group'] == 'Close']['value'].sem(), df.loc[df['group'] == 'Far']['value'].sem()
print df.loc[df['group'] == 'Close']['value'].shape, df.loc[df['group'] == 'Far']['value'].shape
print mannwhitneyu(df.loc[df['group'] == 'Close']['value'], df.loc[df['group'] == 'Far']['value'])
# print pearsonr(df['value'], df['value_dist'])

### SWR recruitment fraction by group

In [ ]:
# SWR Recruitment Fraction all cells
merge_df = soma_responses.merge(activity_dists, on=['roi', 'expt', 'condition', 'mouse_name', 'session'], suffixes=['_swr', '_dist'])
merge_df['value_dist'] = merge_df['peak_value']
merge_df['group'] = merge_df.apply(dist_grp, axis=1)

fig = plt.figure(figsize=(0.8, 1.5), dpi=200)
ax = fig.add_subplot(111)

df = merge_df.loc[(merge_df['condition'] == 2) & (merge_df['n_ripples'] > 0)]
sns.boxplot(y='frac_active', x='group', data=df, order=['Close', 'Far'],
            showcaps=True, showfliers=False, whis=[5, 95], palette=color_dict)
plt.setp(ax.artists, edgecolor='k', lw=1);
plt.setp(ax.lines, color='k', lw=1);
sns.stripplot(x='group', y='frac_active', data=df, order=['Close', 'Far'], palette=color_dict, edgecolor='k', linewidth=0.5, alpha=0.5, size=2)
ax.set_xticklabels(['Near RZ', 'Far'])
ax.set_ylabel('SWR Recruitment Fraction')
ax.set_xlabel('')

clean(ax, offset=4)

print mannwhitneyu(df.loc[df['group'] == 'Close']['frac_active'], df.loc[df['group'] == 'Far']['frac_active'])
print df.loc[df['group'] == 'Close']['frac_active'].mean(), df.loc[df['group'] == 'Far']['frac_active'].mean()
print df.loc[df['group'] == 'Close']['frac_active'].sem(), df.loc[df['group'] == 'Far']['frac_active'].sem()
print df.loc[df['group'] == 'Close']['frac_active'].shape, df.loc[df['group'] == 'Far']['frac_active'].shape
fig.savefig(os.path.join(plotdir, 'all_dist_group_swr_recruitment_peak.svg'))

In [ ]:
a = df.loc[df['group'] == 'Close']['frac_active'].dropna()
b = df.loc[df['group'] == 'Far']['frac_active'].dropna()

print a.shape, a.mean(), a.sem()
print b.shape, b.mean(), b.sem()

In [ ]:
merge_df = soma_responses.merge(pf_dists, on=['roi', 'expt', 'condition', 'mouse_name', 'session'], suffixes=['_swr', '_dist'])
# merge_df['value_dist'] = merge_df['value']
merge_df['value_dist'] = merge_df['peak_value']
merge_df['group'] = merge_df.apply(dist_grp, axis=1)

fig = plt.figure(figsize=(0.8, 1.5), dpi=200)
ax = fig.add_subplot(111)

df = merge_df.loc[(merge_df['condition'] == 2) & (merge_df['n_ripples'] > 0)]
sns.boxplot(y='frac_active', x='group', data=df, order=['Close', 'Far'],
            showcaps=True, showfliers=False, whis=[5, 95], palette=color_dict)
plt.setp(ax.artists, edgecolor='k', lw=1);
plt.setp(ax.lines, color='k', lw=1);
sns.stripplot(x='group', y='frac_active', data=df, order=['Close', 'Far'], palette=colors, edgecolor='k', linewidth=0.5, alpha=0.5, size=2)
ax.set_xticklabels(['Near RZ', 'Far'])
ax.set_ylabel('SWR Recruitment Fraction')
ax.set_xlabel('')

clean(ax, offset=4)

print mannwhitneyu(df.loc[df['group'] == 'Close']['frac_active'], df.loc[df['group'] == 'Far']['frac_active'])
print df.loc[df['group'] == 'Close']['frac_active'].mean(), df.loc[df['group'] == 'Far']['frac_active'].mean()
print df.loc[df['group'] == 'Close']['frac_active'].sem(), df.loc[df['group'] == 'Far']['frac_active'].sem()
print df.loc[df['group'] == 'Close']['frac_active'].shape, df.loc[df['group'] == 'Far']['frac_active'].shape

# fig.savefig(os.path.join(plotdir, 'dist_group_swr_recruitment_peak.svg'))

In [ ]:
ols?

In [ ]:
from statsmodels.formula.api import ols

In [ ]:
formula = 'value ~ C(group) + C(dist_grp) + C(group)*C(dist_grp)'
lm = ols(formula, df).fit()

## SWR Coactivity Index Supp

In [ ]:
# Alternate, stricter definitions
def group_dends(x):
    
    if (x['n_coactive'] > 0) and (x['n_solo'] == 0):
        return 'coactive'
    elif (x['n_solo'] > 0) and (x['n_coactive'] == 0):
        return 'solo'
    elif x['n_solo'] + x['n_coactive'] > 0:
        return 'mixed'
    elif x['n_solo'] + x['n_coactive'] == 0:
        return 'other'
    else:
        print 'WHAT'

def swr_coactivity_index(x):
    
    if x['n_coactive'] + x['n_solo'] == 0:
        return np.nan
    
    return (x['n_coactive'] - x['n_solo']) #/ float(x['n_coactive'] + x['n_solo'])

swr_responses['type'] = swr_responses.apply(group_dends, axis=1)

In [ ]:
gol_paired_br = pair_gol(gol_running_br, same_day=True, next_day=True)
gol_paired_br = gol_paired_br.loc[gol_paired_br['condition_1'] == gol_paired_br['condition_2']]
swr_responses['type'] = swr_responses.apply(group_dends, axis=1)
merge_df = swr_responses.merge(gol_paired_br, on=['mouse_name', 'roi', 'fov', 'session'], suffixes=['_swr', ''])
merge_df['delta'] = merge_df['value_2'] - merge_df['value_1']
# merge_df['sci'] = merge_df.apply(swr_coactivity_index, axis=1)
# merge_df['sci_grp'] = merge_df['sci'].apply(sci_grp)

In [ ]:
merge_df = merge_df.merge(activity_dists, left_on=['mouse_name', 'soma', 'fov', 'session'], right_on=['mouse_name', 'roi', 'fov', 'session'])

In [ ]:
fig = plt.figure(figsize=(1.5, 1.5), dpi=200)
ax = fig.add_subplot(111)

colors = ['#ecd68e', '#f9b0a3', '#cccccc']

merge_df = merge_df.loc[merge_df['type'] != 'mixed']

sns.boxplot(hue='type', x='condition', y='delta', whis=[5, 95], data=merge_df,
            order=[1, 2], hue_order=['coactive', 'solo', 'other'],
            showcaps=True, showfliers=False, palette=colors)
plt.setp(ax.artists, edgecolor='k', lw=1);
plt.setp(ax.lines, color='k', lw=1);
sns.stripplot(x='condition', hue='type', y='delta', data=merge_df, hue_order=['coactive', 'solo', 'other'],
              edgecolor='k', linewidth=0.5, alpha=0.5, palette=colors, size=2, order=[1, 2], dodge=True)

ax.set_ylabel(r'24H $\Delta$ Dendrite-Soma Coupling')
ax.set_xlabel('Condition')


clean(ax, offset=4)

plt.setp(ax.artists, edgecolor='k');
plt.setp(ax.lines, color='k');

ax.axhline(0, color='k', ls='--', lw=1)

ax.set_ylim([-1.1, 1.1])
ax.legend().set_visible(False)

# fig.savefig(os.path.join(plotdir, 'gol_swr_coactivity.svg'))

In [ ]:
cond=2

a = merge_df.loc[(merge_df['type'] == 'coactive') & (merge_df['condition'] == cond)]['delta'].dropna()
b = merge_df.loc[(merge_df['type'] == 'solo') & (merge_df['condition'] == cond)]['delta'].dropna()
c = merge_df.loc[(merge_df['type'] == 'other') & (merge_df['condition'] == cond)]['delta'].dropna()

print f_oneway(a, b, c)

print a.shape[0], b.shape[0], c.shape[0]

print a.mean(), a.sem()
print b.mean(), b.sem()
print c.mean(), c.sem()

print pairwise_tukeyhsd(endog=merge_df.loc[merge_df['condition'] == cond]['delta'],
                        groups=merge_df.loc[merge_df['condition'] == cond]['type'], alpha=0.05)

In [ ]:
def swr_coactivity_normed(x):
    
    if x['n_coactive'] + x['n_solo'] == 0:
        return np.nan
    
    return (x['n_coactive'] - x['n_solo']) / float(x['n_coactive'] + x['n_solo'])

# swr_responses['type'] = swr_responses.apply(group_dends, axis=1)
# merge_df = swr_responses.merge(paired_br, on=['mouse_name', 'roi', 'fov', 'day', 'condition'], suffixes=['_swr', ''])
# merge_df['delta'] = merge_df['value_2'] - merge_df['value_1']

merge_df['scn'] = merge_df.apply(swr_coactivity_normed, axis=1)

fig = plt.figure(figsize=(0.8, 1.5), dpi=200)
ax = fig.add_subplot(111)

colors = ['#00a79d', '#9e1f63']
# sns.lineplot(x='scn', y='delta', hue='day_1', data=merge_df, palette=colors, err_style='bars', ci=68, lw=1, err_kws={'elinewidth':1})
sns.scatterplot(x='scn', y='delta', hue='condition', data=merge_df, palette=colors)

clean(ax, offset=4)
ax.set_ylabel(r'24H $\Delta$ Dendrite-Soma Coupling')
ax.set_xlabel('N Coactive - N Isolated\nN Coactive + N Isolated')

ax.axhline(0, color='k', ls='--')

# ax.set_xticks(range(-1, 4))
# ax.set_xticklabels(['-1-', '0', '1', '2', '3+'])

ax.get_legend().set_visible(False)

ax.set_ylim([-1.1, 1.1])
fig.savefig(os.path.join(plotdir, 'swr_index_by_condition.svg'))

In [ ]:
test_df = merge_df.loc[merge_df['type'] != 'other']
spearmanr(test_df.loc[test_df['condition'] == 1]['scn'], test_df.loc[test_df['condition'] == 1]['delta'])

In [ ]:
spearmanr(test_df.loc[test_df['condition'] == 2]['scn'], test_df.loc[test_df['condition'] == 2]['delta'])

In [ ]:
fig = plt.figure(figsize=(0.8, 1.5), dpi=200)
ax = fig.add_subplot(111)
ax.hist(merge_df['scn'].dropna())
ax.set_xlabel('N Coactive - N Isolated\nN Coactive + N Isolated')
ax.set_ylabel('N Dendrites')
clean(ax)
fig.savefig(os.path.join(plotdir, 'sci_distribution.svg'))

## Compute Everything

In [ ]:
data_path = '/home/sebi/data/dends'

# gol_running_bsp = da.branch_spike_prevalence(gol_grp, signal=None, running_only=True)
gol_in_pf_bsp = da.branch_spike_prevalence(gol_grp, signal=None, running_only=True, in_field_only=True)
# gol_nonrunning_bsp = da.branch_spike_prevalence(gol_grp, signal=None, non_running_only=True, non_ripple_only=True)
# gol_swr_bsp = da.branch_spike_prevalence(gol_grp, signal=None, non_running_only=True, ripple_only=True)


# gol_running_br = da.branch_recruitment(gol_grp, signal=None, running_only=True)
# gol_nonrunning_br = da.branch_recruitment(gol_grp, signal=None, non_running_only=True, non_ripple_only=True)
# gol_swr_br = da.branch_recruitment(gol_grp, signal=None, non_running_only=True, ripple_only=True)

# gol_running_spikes = da.branch_spikes(gol_grp, signal=None, running_only=True)
# gol_nonrunning_spikes = da.branch_spikes(gol_grp, signal=None, non_running_only=True, non_ripple_only=True)
# gol_swr_spikes = da.branch_spikes(gol_grp, signal=None, non_running_only=True, ripple_only=True)

# Save

# gol_running_bsp.to_pickle(os.path.join(data_path, 'gol_running_bsp.pkl'))
# gol_nonrunning_bsp.to_pickle(os.path.join(data_path, 'gol_nonrunning_bsp.pkl'))
# gol_swr_bsp.to_pickle(os.path.join(data_path, 'gol_swr_bsp.pkl'))
# gol_in_pf_bsp.to_pickle(os.path.join(data_path, 'gol_in_pf_bsp.pkl'))

# gol_running_br.to_pickle(os.path.join(data_path, 'gol_running_br.pkl'))
# gol_nonrunning_br.to_pickle(os.path.join(data_path, 'gol_nonrunning_br.pkl'))
# gol_swr_br.to_pickle(os.path.join(data_path, 'gol_swr_br.pkl'))

# gol_running_spikes.to_pickle(os.path.join(data_path, 'gol_running_spikes.pkl'))
# gol_nonrunning_spikes.to_pickle(os.path.join(data_path, 'gol_nonrunning_spikes.pkl'))
# gol_swr_spikes.to_pickle(os.path.join(data_path, 'gol_swr_spikes.pkl'))

In [ ]:
# Fraction Licks
lick_dist = ra.lick_to_reward_distance(gol_grp)
lick_dist.to_pickle(os.path.join(data_path, 'lick_distance.pkl'))

In [ ]:
def corr_dist_grp(x):
    if x['corr'] > 0.5:
#     if x['value'] > 0.3:
        if x['value_dist'] < .167:
            return 'High RZ'
        elif ['value_dist'] > .33:
            return 'High Far'
    else:
        if x['value_dist'] < .167:
            return 'Low RZ'
        elif ['value_dist'] > .33:
            return 'Low Far'

def dist_grp(x):
    # Split distances into thirds
    if x['value_dist'] <= 0.167:
        return 'Close'
    elif x['value_dist'] > .33:
        return 'Far'
    else:
        return 'Mid'

def pair_gol(df, same_day=True, next_day=False):
    sessions = []
    if same_day:
        sessions += [1, 3, 5, 7]
    if next_day:
        sessions += [2, 4, 6]
    
    paired_dfs = []
    for s1 in sessions:
        s2 = s1 + 1
        
        s1_df = df.loc[df['session'] == s1]
        s2_df = df.loc[df['session'] == s2]
        
        merged_df = s1_df.merge(s2_df, on=['mouse_name', 'roi', 'fov'], suffixes=['_1', '_2'])
        merged_df['session'] = s1
        paired_dfs.append(merged_df)
    
    return pd.concat(paired_dfs)

In [ ]:
pfs = gol_grp.pfs(roi_filter=lambda x: '_' not in x.label)

In [ ]:
n = 0
m = 0
for expt in gol_grp:
    epfs = pfs[expt]
    n += len([x for x in epfs if x])
    m += len(epfs)

In [ ]:
def dist(a, b):
    x = a - b
    if x <= -50:
        x += 100
    elif x > 50:
        x -= 100
    return x


def pf_centroid_dists(gol_grp, roi_filter=None):
    # Distance of each pf centroid to RZ
    if not roi_filter:
        roi_filter = lambda x: '_' not in x.label

    data = gol_grp.data(roi_filter=roi_filter)
    pfs = gol_grp.pfs(roi_filter=roi_filter)

    data_list = []
    
    rz1_dict = {}
    for expt in gol_grp:
        if expt.condition == 2:
            continue
        mouse_name = expt.parent.mouse_name
        fov = expt.get('uniqueLocationKey')
        rz1_dict[(mouse_name, fov)] = expt.rewardPositions(units='normalized')[0] * 100
        

    for expt in gol_grp:

        centroids = pca.calcCentroids(data[expt], pfs[expt], returnAll=False)

        reward_pos = expt.rewardPositions(units='normalized')[0] * 100
        rois = expt.rois(label='mergedmerged', roi_filter=roi_filter)
        
        if expt.condition == 1:
            old_reward = np.nan
        else:
            mouse_name = expt.parent.mouse_name
            fov = expt.get('uniqueLocationKey')
            old_reward = rz1_dict[(mouse_name, fov)]

        for roi, centroid, tc in zip(rois, centroids, data[expt]):

            if len(centroid) == 0:
                continue
            else:
                centroid = centroid[0]
            
            peak = np.argmax(tc)

            data_list.append({'expt': expt.trial_id,
                              'mouse_name': expt.parent.mouse_name,
                              'session': expt.session,
                              'condition': expt.condition,
                              'day': expt.day,
                              'roi': roi.label,
                              'fov': expt.get('uniqueLocationKey'),
                              'centroid': centroid,
                              'reward_pos': reward_pos,
                              'peak': peak,
                              'value': np.abs(dist(reward_pos, centroid)) / 100.,
                              'peak_value': np.abs(dist(reward_pos, peak)) / 100.,
                              'old_value': np.abs(dist(old_reward, centroid)) / 100.,
                              'old_peak_value': np.abs(dist(old_reward, peak)) / 100.})

    pf_dists = pd.DataFrame(data_list)
    return pf_dists

pf_dists = pf_centroid_dists(gol_grp)

In [ ]:
def indicator(pf, nbins=100):
    
    pf_indicator = np.zeros((nbins,), dtype=bool)
    start = int(pf[0] * nbins)
    stop = int(pf[1] * nbins)
    if start < stop:
        pf_indicator[start:stop] = 1
    else:
        pf_indicator[start:] = 1
        pf_indicator[:stop] = 1
    return pf_indicator

def tc_correlation(cell_set):
    data_list = []
    for cell in cell_set:
        
        mouse_name = cell.parent_experiment.parent.mouse_name
        fov = cell.parent_experiment.get('uniqueLocationKey')
        day = cell.parent_experiment.get('day')
        condition = cell.parent_experiment.get('condition')
        session = cell.parent_experiment.get('session')
        tid = cell.parent_experiment.trial_id
        
        belt_len = cell.parent_experiment.belt_length / 1000.
        
        soma_tc = cell.soma.data()
        soma_peak = np.argmax(soma_tc)
        
        if cell.soma.pfs():
            soma_pf = True
            soma_indicator = indicator(cell.soma.pfs()[0])
        else:
            soma_pf = False
            soma_indicator = np.zeros((100,), dtype=bool)
        
        dend_tcs = [d.data() for d in cell.dendrites]
        
        for d, d_tc in zip(cell.dendrites, dend_tcs):
            
            corr = pearsonr(d_tc, soma_tc)[0]
            dend_peak = np.argmax(d_tc)
            
            peak_dist = dist(soma_peak, dend_peak)
#             peak_dist *= belt_len
            peak_dist /= 100.
            
            if d.pfs():
                dend_pf = True
                dend_indicator = indicator(d.pfs()[0])
            else:
                dend_pf = False
                dend_indicator = np.zeros((100,), dtype=bool)
            
            pf_overlap = np.any(soma_indicator & dend_indicator)
            
            if soma_pf and dend_pf and (not pf_overlap) and (peak_dist == 0):
                print tid
                print cell.label
                print cell.soma.pfs()
                print d.label
                print d.pfs()
            
            data_list.append({'mouse': mouse_name,
                              'fov': fov,
                              'day': day,
                              'condition': condition,
                              'session': session,
                              'expt': tid,
                              'soma': cell.soma.label,
                              'dend': d.label,
                              'order': d.order,
                              'corr': corr, 
                              'pf': soma_pf,
                              'dend_pf': dend_pf,
                              'pf_overlap': pf_overlap,
                              'peak_dist': peak_dist,
                              'abs_dist': np.abs(peak_dist)})

    return pd.DataFrame(data_list)

In [ ]:
tuning_df = tc_correlation(cell_set)

In [ ]:
dend_pf_dists = pf_centroid_dists(gol_grp, roi_filter=lambda x: '_' in x.label)

In [ ]:
def convert_centroid(x):
    """Convert centroid from complex value to position.

    Take angle and convert from radians.
    """
    return ((np.angle(x) % (2 * np.pi))) / (2 * np.pi) * 100

def activity_centroid_dists(gol_grp):
    soma_filter = lambda x: '_' not in x.label

    data_list = []

    centroids = pca.calc_activity_centroids(gol_grp, roi_filter=soma_filter)

    pfs = gol_grp.pfs(roi_filter=soma_filter)
    data = gol_grp.data(roi_filter=soma_filter)

    for expt in gol_grp:

        trial_id = expt.trial_id
        mouse_name = expt.parent.mouse_name

        rois = expt.rois(roi_filter=soma_filter)

        reward_pos = expt.rewardPositions(units='normalized')[0] * 100

        for roi, pf, centroid, tc in zip(rois, pfs[expt], centroids[expt], data[expt]):
            if centroid:

                centroid_pos = convert_centroid(centroid)
                peak = np.argmax(tc)

                if pf:
                    is_pf = True
                else:
                    is_pf = False

                data_list.append({'mouse_name': mouse_name,
                                  'expt': trial_id,
                                  'session': expt.session,
                                  'condition': expt.condition,
                                  'day': expt.day,
                                  'roi': roi.label,
                                  'fov': expt.get('uniqueLocationKey'),
                                  'centroid': centroid_pos,
                                  'dist': dist(centroid_pos, reward_pos) / 100.,
                                  'value': np.abs(dist(centroid_pos, reward_pos)) / 100.,
                                  'peak': peak,
                                  'peak_value': np.abs(dist(peak, reward_pos)) / 100.,
                                  'pf': is_pf})
                
    centroid_dists = pd.DataFrame(data_list).dropna()
    return centroid_dists

activity_dists = activity_centroid_dists(gol_grp)
paired_activities = pair_gol(activity_dists, same_day=True, next_day=True)

In [ ]:
pf_dists.to_pickle(os.path.join(data_path, 'pf_dists.pkl'))
paired_pf_dists.to_pickle(os.path.join(data_path, 'paired_pf_dists.pkl'))
activity_dists.to_pickle(os.path.join(data_path, 'activity_dists.pkl'))
paired_activities.to_pickle(os.path.join(data_path, 'paired_activities.pkl'))

In [ ]:
gol_swr_responses = da.swr_activity(gol_grp)
soma_responses = gol_swr_responses.copy()
soma_responses['n_active'] = soma_responses['n_missed'] + soma_responses['n_coactive']
soma_responses = soma_responses.groupby(['condition', 'day', 'expt', 'fov', 'mouse_name', 'soma', 'session'], as_index=False).mean()
soma_responses['roi'] = soma_responses['soma']
soma_responses['frac_active'] = soma_responses['n_active'] / soma_responses['n_ripples']

In [ ]:
gol_swr_responses.to_pickle(os.path.join(data_path, 'gol_swr_responses.pkl'))

In [ ]:
gol_grp[0].get('tSeriesDirectory')

In [ ]:
running_freq = da.frequency(gol_grp, signal=None, running_only=True)
all_freq = da.frequency(gol_grp, signal=None)

In [ ]:
swr_responses = da.swr_activity(gol_grp)